In [15]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [16]:
df = pd.read_csv("abalone.csv")
print(df.head())

df["Sex"] = df["Sex"].replace({"M":0, "F":1, "I":2})


   Rings Sex  Length  Diameter  Height  WholeWeight  ShuckedWeight  \
0     15   M   0.455     0.365   0.095       0.5140         0.2245   
1      7   M   0.350     0.265   0.090       0.2255         0.0995   
2      9   F   0.530     0.420   0.135       0.6770         0.2565   
3     10   M   0.440     0.365   0.125       0.5160         0.2155   
4      7   I   0.330     0.255   0.080       0.2050         0.0895   

   VisceraWeight  ShellWeight  
0         0.1010        0.150  
1         0.0485        0.070  
2         0.1415        0.210  
3         0.1140        0.155  
4         0.0395        0.055  


/var/folders/64/_d7wbs111535lmdv26qfwp000000gn/T/ipykernel_3053/1162198912.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Sex"] = df["Sex"].replace({"M":0, "F":1, "I":2})


In [17]:
Y = df["Rings"].values
X = df.iloc[:, 1:]

In [18]:
scaler = StandardScaler()
X_normalize = scaler.fit_transform(X)

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_normalize, Y,train_size=0.70, test_size=0.30, random_state=42
)

print(f"This is the shape of X_train: {X_train.shape}")
print(f"This is the shape of Y_train: {Y_train.shape}")
print(f"This is the shape of X_test: {X_test.shape}")
print(f"This is the shape of Y_Test: {Y_test.shape}")

This is the shape of X_train: (2923, 8)
This is the shape of Y_train: (2923,)
This is the shape of X_test: (1254, 8)
This is the shape of Y_Test: (1254,)


In [20]:
#Convert tensors of Pytorch(the numpy.array of tensorflow)

x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(Y_train, dtype=torch.float32)
x_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(Y_test, dtype=torch.float32)

#Making the DataSet , because needs to do it
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [27]:
#This is defining the function like keras or TensorFlow
class FNN_Model_Abalone(nn.Module):
    def __init__(self, input_size, output_size):
        super(FNN_Model_Abalone, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, output_size)
        )
        
    def forward(self, x):
        return self.network(x)

In [28]:
#This is the FNN that has the X that are all the input size and the output size
model = FNN_Model_Abalone(x_train.shape[1], 1)
criterion = nn.MSELoss()
optimzer = optim.AdamW(model.parameters(), lr=0.0001)

In [29]:
EPOCHS = 300
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0.0
    
    for batch_x, batch_y in train_loader:
        optimzer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimzer.step()
        epoch_loss += loss.item()
        
    if(epoch +1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {epoch_loss/len(train_loader):.4f}")

        

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([43])) that is different to the input size (torch.Size([43, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [50/300], Loss: 10.7247
Epoch [100/300], Loss: 10.5096
Epoch [150/300], Loss: 10.5500
Epoch [200/300], Loss: 10.5151
Epoch [250/300], Loss: 10.4897
Epoch [300/300], Loss: 10.5258


In [30]:
model.eval()
with torch.no_grad():
    y_pred = model(x_test).numpy()
    
mean_squared_error_ = mean_squared_error(y_test.numpy(), y_pred)
rmse = np.sqrt(mean_squared_error_)
mean_absolute_error_ = mean_absolute_error(y_test.numpy(), y_pred)

print("\n Results")
print(f"MSE: {mean_squared_error_:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mean_absolute_error_:.4f}")


 Results
MSE: 9.9798
RMSE: 3.1591
MAE: 2.3391
